# <center>**Data Preprocessing**</center>
---
---

### Import the necessary library

In [ ]:
import pandas as pd 
import numpy as np 
import re

### Loading the dataset and obtaining more information about it

In [2]:
Data =  pd.read_csv("Cars.csv" , encoding='ISO-8859-1' )


In [3]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45940 entries, 0 to 45939
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Make              45940 non-null  object
 1   Model             45940 non-null  object
 2   Year              45940 non-null  object
 3   Transmission      45940 non-null  object
 4   Fuel              45940 non-null  object
 5   Condition         45518 non-null  object
 6   Kilometers        45940 non-null  object
 7   Paint             44459 non-null  object
 8   Interior Options  45940 non-null  object
 9   Exterior Options  45940 non-null  object
 10  Jayed             16317 non-null  object
 11  Price             45940 non-null  object
dtypes: object(12)
memory usage: 4.2+ MB


In [4]:
Data.describe()

,Make,Model,Year,Transmission,Fuel,Condition,Kilometers,Paint,Interior Options,Exterior Options,Jayed,Price
count,45940,45940,45940,45940,45940,45518,45940,44459,45940,45940,16317,45940
unique,87,896,55,2,6,2,23,4,22476,7033,185,839
top,Hyundai,Avante,2017,Automatic,Gasoline,Used,0,Original Paint,Air Condition,Rim Wheels,??? ????,"3,500 JOD"
freq,10805,3090,2224,33940,29707,44204,7248,19774,1488,5226,5218,811


---
## Data Cleansing 
### Drop Duplicates

In [5]:
Data = Data.drop_duplicates()
Data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34715 entries, 0 to 45939
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Make              34715 non-null  object
 1   Model             34715 non-null  object
 2   Year              34715 non-null  object
 3   Transmission      34715 non-null  object
 4   Fuel              34715 non-null  object
 5   Condition         34346 non-null  object
 6   Kilometers        34715 non-null  object
 7   Paint             33614 non-null  object
 8   Interior Options  34715 non-null  object
 9   Exterior Options  34715 non-null  object
 10  Jayed             12520 non-null  object
 11  Price             34715 non-null  object
dtypes: object(12)
memory usage: 3.4+ MB


---

### Include the brands with a count of more than 30 in the dataset.

In [6]:
excluded_makes = ["Daihatsu", "Lincoln", "Skoda", "Changan", "MG", 
                  "Cadillac", "Samsung", "Saab", "Porsche", 
                  "Chery", "Chrysler", "Infiniti", "SEAT", 
                  "Jaguar", "Subaru"]

# ~ stands for Not
Data = Data[~Data['Make'].isin(excluded_makes)]


---
### If the value of the kilometers is not 0, it should be converted to "Used".      
### for cars have Null condition and have kilometers equal to 0, but the condition is not used should be marked as "New".

In [7]:
Data['Condition'] = np.where(Data['Kilometers'] != '0', 'Used', Data['Condition'])
Data['Condition'] = np.where((Data['Kilometers'] == '0') & (Data['Condition'] != "Used") , "New" , Data['Condition']) 
Data['Condition'].value_counts()

Condition
Used    33568
New       380
Name: count, dtype: int64

---
### Split the kilometers coulmn into two columns so that we can calculate the average kilometers and convert it to float numbers.

In [8]:
Data[['Kilometers1', 'Kilometers2']] = Data['Kilometers'].str.split('-' , expand=True)

Data['Kilometers1'] = Data['Kilometers1'].str.replace(',', '').astype(float)
Data['Kilometers2'] = Data['Kilometers2'].str.replace(',', '').astype(float)


In [9]:
Data = Data.drop("Kilometers", axis =  1)

Data['Avarege Kilometers'] = np.where(Data['Kilometers1'] == 200000, 
                            200000, 
                            Data['Kilometers1'] + Data['Kilometers2'] / 2)

# Fill the null with 0 for new cars
Data = Data.drop(["Kilometers1", "Kilometers2"] , axis =1 )
Data['Avarege Kilometers'] = Data['Avarege Kilometers'].fillna(0)


In [10]:
Data['Avarege Kilometers'] = np.where(
    (Data['Avarege Kilometers'] == 0.0) & (Data['Condition'] == 'Used'),
    109999.5,
    Data['Avarege Kilometers']
)

---
### Handle the price coulmn convert it from string to integer number 

In [11]:
# Function to remove currency symbols and convert to float
def clean_currency(value):
    # Remove non-numeric characters using regex
    value = re.sub(r'[^\d.]', '', value)
    return int(value) if value else None

# Apply the function to the 'Amount' column
Data['Price'] = Data['Price'].apply(clean_currency)

Data.head(1)

,Make,Model,Year,Transmission,Fuel,Condition,Paint,Interior Options,Exterior Options,Jayed,Price,Avarege Kilometers
0,Mercedes Benz,Vito,2007,Automatic,Diesel,Used,Original Paint,"Air Condition , ABS Brakes , Electric Windows ...","Rim Wheels , Rear Sensors , Electric Mirrors","??? ????, ??? ????",9350,200000.0


---
### Filling the Nulls in Paint column with "Original Paint" if the Condition is New Otherwise "Other"

In [12]:
Data['Paint'] = np.where(Data['Paint'].isna() & (Data['Condition'] == 'New'), 'Original Paint', Data['Paint'])

Data['Paint'] = Data['Paint'].fillna('Other')


In [13]:
Data.isna().sum()

Make                      0
Model                     0
Year                      0
Transmission              0
Fuel                      0
Condition                 0
Paint                     0
Interior Options          0
Exterior Options          0
Jayed                 21677
Price                     0
Avarege Kilometers        0
dtype: int64

---
### For Interior and Exterior Options I Convert the 10 for Interior and 7 for Exterior most significant Options in the car 
### price "Based in ChatGBT" to new coulmns and give it a True , False value if the option contains or not 

In [14]:
Options = ["Leather Seats" , "Navigation System" , "Heated Seats" , "Air Condition" , "Bluetooth", "Cruise Control" ,
           "Traction Control" , "Touch Screen" , "Android Auto" , "Heated Steering Wheel"]
Options2 = ["Panoramic Roof" , "Sports Package" , "Keyless Entry" , "LED Lights" ,"Rim Wheels" ,"Xenon Lights" ,"Sunroof"]

for O in Options : 
    Data[O] = Data['Interior Options'].str.contains(O).astype(bool)

for O in Options2 : 
    Data[O] = Data['Exterior Options'].str.contains(O).astype(bool)

Data.columns

Index(['Make', 'Model', 'Year', 'Transmission', 'Fuel', 'Condition', 'Paint',
       'Interior Options', 'Exterior Options', 'Jayed', 'Price',
       'Avarege Kilometers', 'Leather Seats', 'Navigation System',
       'Heated Seats', 'Air Condition', 'Bluetooth', 'Cruise Control',
       'Traction Control', 'Touch Screen', 'Android Auto',
       'Heated Steering Wheel', 'Panoramic Roof', 'Sports Package',
       'Keyless Entry', 'LED Lights', 'Rim Wheels', 'Xenon Lights', 'Sunroof'],
      dtype='object')

In [15]:
Data = Data.drop(["Interior Options", "Exterior Options"] , axis =1 )


In [16]:
Data.head(1)

,Make,Model,Year,Transmission,Fuel,Condition,Paint,Jayed,Price,Avarege Kilometers,...,Touch Screen,Android Auto,Heated Steering Wheel,Panoramic Roof,Sports Package,Keyless Entry,LED Lights,Rim Wheels,Xenon Lights,Sunroof
0,Mercedes Benz,Vito,2007,Automatic,Diesel,Used,Original Paint,"??? ????, ??? ????",9350,200000.0,...,True,False,False,False,False,False,False,True,False,False


In [17]:
Data.to_csv(r"C:\Users\user\OneDrive\Desktop\Car Price Prediction\Data Post.csv")